Importing necessary libraries

In [16]:
import pandas as pd

Map continent data with yield.csv

In [17]:
yield_data = pd.read_csv('original_data/yield.csv')

continent_data = pd.read_csv('processed_data/countries_by_continent_sorted.csv')

combined_yield = pd.merge(yield_data, continent_data, how='left', left_on='Area', right_on='Country')

combined_yield = combined_yield.drop(columns=['Country']).rename(columns={'Continent': 'Country_Continent'})

combined_yield.to_csv('processed_data/yield_continent.csv', index=False)

Map pesticides data with the updated yield

In [18]:
yield_data = pd.read_csv('processed_data/yield_continent.csv')

pesticides_data = pd.read_csv('original_data/pesticides.csv')

combined_yield = pd.merge(yield_data, pesticides_data, how='left', on=['Area', 'Year'])

combined_yield.to_csv('processed_data/yield_continent_pesticides.csv', index=False)


Map rainfall data with the updated yield

In [19]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides.csv')
 
rainfall_data = pd.read_csv('original_data/rainfall.csv')
 
combined_yield = pd.merge(yield_data, rainfall_data, how='left', on=['Area', 'Year'])
 
combined_yield.to_csv('processed_data/yield_continent_pesticides_rainfall.csv', index=False)


Map temperature data with the updated yield

In [20]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides_rainfall.csv')
 
temperature_data = pd.read_csv('original_data/temp.csv')

combined_yield = pd.merge(yield_data, temperature_data, how='left', left_on=['Area', 'Year'], right_on=['country', 'year'])

combined_yield = combined_yield.drop(columns=['country', 'year'])

combined_yield.to_csv('processed_data/combined_raw_data.csv', index=False)


/var/folders/zc/ndxw3gq100s4y4g6l1dmhflh0000gn/T/ipykernel_81162/119835002.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  yield_data = pd.read_csv('processed_data/yield_continent_pesticides_rainfall.csv')


Remove unecessary columns

In [21]:
file_path = 'processed_data/combined_raw_data.csv'
 
columns_to_delete = ['Domain Code', 'Domain_x', 'Area Code', 'Element_x', 'Element Code', 'Year Code', 'Domain_y', 'Element_y', ]

df = pd.read_csv(file_path)

df = df.drop(columns=columns_to_delete, errors='ignore')

df.to_csv(file_path, index=False)

/var/folders/zc/ndxw3gq100s4y4g6l1dmhflh0000gn/T/ipykernel_81162/544248313.py:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
